# Activate your environment as Kernel

In [5]:
!conda init
!conda activate my_llms
!python -m ipykernel install --user --name=my_llms --display-name="Python (my_llms)"

no change     /opt/anaconda3/condabin/conda
no change     /opt/anaconda3/bin/conda
no change     /opt/anaconda3/bin/conda-env
no change     /opt/anaconda3/bin/activate
no change     /opt/anaconda3/bin/deactivate
no change     /opt/anaconda3/etc/profile.d/conda.sh
no change     /opt/anaconda3/etc/fish/conf.d/conda.fish
no change     /opt/anaconda3/shell/condabin/Conda.psm1
no change     /opt/anaconda3/shell/condabin/conda-hook.ps1
no change     /opt/anaconda3/lib/python3.12/site-packages/xontrib/conda.xsh
no change     /opt/anaconda3/etc/profile.d/conda.csh
no change     /Users/gulati/.bash_profile
No action taken.

CondaError: Run 'conda init' before 'conda activate'

Installed kernelspec my_llms in /Users/gulati/Library/Jupyter/kernels/my_llms


### GOAL:
Build a tool that infers important and relevant information for a product, creates a marketing brochure for the product, and translates the brochure to another language (based on user input).

In [1]:
# imports
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
import json
from openai import OpenAI
import os
import requests
import ollama
import signal


In [2]:
# define function for timeout for user input

def timeout_handler(signum, frame):
    raise TimeoutError

# Set the signal handler and a 5-second alarm
signal.signal(signal.SIGALRM, timeout_handler)
signal.alarm(5)

0

In [4]:
!conda list ollama
!ollama pull llama3.2

# packages in environment at /opt/anaconda3/envs/llms:
#
# Name                    Version                   Build  Channel
ollama                    0.4.7                    pypi_0    pypi
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [5]:
# constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [6]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print('API key looks fine')
else:
    print('Somehting is wrong with the API key.')

openai = OpenAI()

API key looks fine


In [7]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent contents of a Website
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
ed = Website("https://www.apple.com/apple-watch-series-10/")
ed.text

"Apple\nStore\nMac\niPad\niPhone\nWatch\nVision\nAirPods\nTV & Home\nEntertainment\nAccessories\nSupport\n0\n+\nApple Watch Series 10\nLocal Nav Menu\nLocal Nav Menu\nOverview\nWhy Apple Watch\nTech Specs\nBuy\nApple Watch Series 10\nGet 3 months of Fitness+ free when you buy any Apple\xa0Watch.\n1\nShop Apple Watch\nApple Watch Series 10\nThinstant\nclassic.\n*\nBuy\nGet the highlights.\nWatch the\nApple\xa0Watch\xa0Series\xa010\nfilm\nItem 1\nItem 2\nItem 3\nItem 4\nItem 5\nThe thinnest Apple\xa0Watch ever, with our biggest display.\n3\nGet invaluable health insights, including sleep apnea notifications.\n5\nEnhance your aquatic adventures with depth and water temperature sensors.\nFastest-charging Apple\xa0Watch ever.\n4\nGet 80% battery in 30 minutes.\nMotivation on your wrist. Apple\xa0Watch inspires you to stay active, day after\xa0day.\nMonumental display.\nSeries 10 is a major milestone for Apple\xa0Watch. It features our\nbiggest and most advanced display yet\n,\n3\nshowing mo

In [9]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    return result

In [10]:
def get_brochure_user_prompt(product_name, url):
    user_prompt = f"You are looking at a product called: {product_name}\n"
    user_prompt += f"Here are the contents of its landing page; use this information to build a short brochure of the product in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [11]:
# System Prompt

system_prompt = """
You are provided with product information found on a webpage. \
You are able to decide which of the text would be most relevant to include in a marketing brochure about the product for prospective customers, \
such as specifications, uniqueness, price, etc. \
Respond in Markdown.
"""

## With GPT

In [12]:
# Funtion for translating brochure to another language
def translate_brochure(info, language, model):
    print("#" * 50)
    print("#" * 20, f"{language} Version",  "#" * 20)
    system_prompt2 = f"""
    Translate the following marketing brochure into {language} language.
    Respond in Markdown.
    """
    messages = [
                {"role": "system", "content": system_prompt2},
                {"role": "user", "content": info}
            ]
    if model==1:
        response = openai.chat.completions.create(
            model=MODEL_GPT,
            messages = messages,
        )
        result = response.choices[0].message.content
    else:
        response = ollama.chat(model=MODEL_LLAMA, messages=messages)
        result = response['message']['content']
    display(Markdown(result))

In [ ]:
# Function for creating brochure
# Parameters:
#   product_name: Name of the product
#   url: website url of the product
#   model: GPT4 or Ollama; 1 for GPT and 0 for Ollama

def create_brochure(product_name, url, model):
    try:
        language = input("Enter language for translation (leave blank if no translation): ")
    except TimeoutError:
        language=''
    messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": get_brochure_user_prompt(product_name, url)}
            ]
    if model==1:
        response = openai.chat.completions.create(
            model=MODEL_GPT,
            messages=messages,
        )
        result = response.choices[0].message.content
    elif model==0:
        response = ollama.chat(model=MODEL_LLAMA, messages=messages)
        result = response['message']['content']
    else:
        raise ValueError("Unknown model. Please choose 1 for GPT and 0 for Ollama")

    display(Markdown(result))
    if language:
        translate_brochure(result, language, model)

In [14]:
create_brochure("Apple Watch Series 10", "https://www.apple.com/apple-watch-series-10/",0)

**Apple Watch Series 10 Brochure**
=====================================

**Introduction**

Stay connected, motivated, and informed with the latest Apple Watch Series 10. Designed for a healthier you, this watch offers cutting-edge technology, sleek design, and endless possibilities.

**Key Features**

### **Display**

* Largest screen area of any Apple Watch
* Brighter display when viewed from an angle
* Wide-angle OLED Display for easier reading

### **Design**

* Thinnest Apple Watch ever
* Jet Black aluminum case with a 30-step anodization process
* Three polished titanium finishes available (Slate, Natural, Gold)

### **Health Insights**

* Advanced sensors track sleep apnea notifications and health metrics
* ECG app generates an electrocardiogram similar to a single-lead reading
* Cycle Tracking app uses temperature data for family planning

### **Fitness**

* Motivates you to stay active with fitness goals and tracking features
* Water temperature sensor and depth sensor for swimming and snorkeling enthusiasts
* Activity app tracks workouts, including running, strength training, Pilates, and dance

**Technical Specifications**

* Processor: New design with improved performance and efficiency
* Battery Life: Up to 80% battery in 30 minutes
* Display Size: Largest screen area of any Apple Watch
* Weight: Almost 20% less than Series 9 in stainless steel

**Get Ready to Upgrade Your Fitness Journey**

Experience the power of Apple Watch Series 10. Get 3 months of Fitness+ free with your purchase, and discover a healthier you.

[Buy Now](insert link)

###

##################################################
#################### French Version ####################


**Apple Watch Série 10 - Brochure**
=====================================

**Introduction**

Restez connecté, motivé et informé avec la dernière Apple Watch Série 10. Conçue pour un vous plus sain, cette montre offre une technologie de pointe, une conception élégante et des possibilités infinies.

**Caractéristiques clés**

### **Affichage**

* Superficie d'écran la plus grande de toute montre Apple Watch
* Affichage plus brillant lorsqu'il est regardé sous un angle
* Écran OLED avec angle large pour une lecture plus facile

### **Conception**

* Thinner que la Série 9
* Cas en aluminium jet black avec un processus d'anodisation à 30 étapes
* Trois finitions de titan poli disponibles (Slate, Naturel, Or)

### **Insights sur la santé**

* Sensoryurs avancés pour détecter les notifications d'apnée et les métriques de santé
* Application ECG qui génère un électrocardiogramme similaire à une lecture de single-lead
* Application Cycle Tracking qui utilise des données de température pour le planification familiaux

### **Fitness**

* Incitation à rester actif avec des objectifs et des fonctionnalités de suivi
* Sensor d'eau pour la température et le profondeur pour les amateurs de natation et de plongée
* Application Activité qui suit les exercices, y compris la course, la renforcement musculaire, le Pilates et la danse

**Spécifications techniques**

* Procédure : Nouvelle conception avec une performance et une efficacité améliorées
* Durée de batterie : Jusqu'à 80% de capacité en 30 minutes
* Taille d'écran : Superficie d'écran la plus grande de toute montre Apple Watch
* Poids : Presque 20% moins lourd que la Série 9 en acier inoxydable

**Prêt à mettre à jour votre parcours de fitness**

Expérimenziez la puissance de la montre Apple Watch Série 10. Getz trois mois de Fitness+ gratuit avec votre achat et découvrez un vous plus sain.

[Commandez maintenant](insert link)